In [ ]:
import collections
import itertools
import glob
import json
import numpy as np

from sklearn.metrics import cohen_kappa_score as kappa
from pyannote.core import Segment
from pygamma_agreement import (CombinedCategoricalDissimilarity,
                               Continuum,
                               show_alignment,
                               show_continuum)

def get_metadata(annotation, key):
    return annotation["metadata"][key]

def get_pair_identifier(anno1, anno2):
    assert get_metadata(anno1, "review_id") == get_metadata(anno2, "review_id")    
    anno_names_in_order = sorted([get_metadata(anno1, "annotator"),
                                  get_metadata(anno2, "annotator")])
    return "|".join([get_metadata(anno1, "review_id")] + anno_names_in_order)

In [ ]:
# === Helpers for gamma ===

def get_contiguous_segments(aligned_indices):
    """
        Segments a list of indices into a list of lists, where each sub-list contains a contiguous subsequence of indices.
    """
    assert aligned_indices == list(sorted(aligned_indices))
    segments = []
    l = list(aligned_indices)
    current_segment = []
    while l:
        k = l.pop(0)
        if current_segment:
            if k == current_segment[-1] + 1:
                current_segment.append(k)
            else:
                segments.append(current_segment)
                current_segment = [k]
        else:
            current_segment = [k]
    segments.append(current_segment)
    return segments

print("Testing get_contiguous_segments")
tgcs_input = [1,2,3,6,7,9,11,12]
print("Input:", tgcs_input)
print("Output:", get_contiguous_segments(tgcs_input))

def get_continuum_items_and_categories(rebuttal_sentences, annotator, key):
    """
        Given rebuttal sentences and the annotator name, produces items for continuum for gamma calculation
        
        Returns 3-tuples, of the format:
            (<annotator_id>, Segment(<segment_start>, <segment_exclusive_end>), <rebuttal_label>) # TODO(TJO): Is exclusive end correct?
        as well as a set of all possible rebuttal labels for this example # TODO(TJO): Does it matter if it's all possible labels or all labels which are present?
    """
    categories = set()
    new_continuum_items = []
    for i, sentence in enumerate(rebuttal_sentences):
        category = sentence[key]
        align_type, align_indices = sentence["alignment"]
        if align_indices is not None:
            segments = get_contiguous_segments(align_indices) # TODO(TJO): Is this the right way to get segments from alignments?
            for segment in segments:
                new_continuum_items.append((annotator, Segment(segment[0], segment[-1] + 1), category))
                categories.add(category)
    
    return new_continuum_items, categories

In [ ]:
# === Helpers for kappa ===

REV_LABELS = "coarse fine asp pol".split()
REB_LABELS = "coarse fine".split()

LABEL_TYPES = ["rev_"+label
               for label in REV_LABELS] + ["reb_"+label
                for label in REB_LABELS]
                                  
def get_label_lists(annotation):
    
    label_lists = {
        label:[] for label in LABEL_TYPES
    }
    
    for rev_label in REV_LABELS:
        for sentence in annotation["review_sentences"]:
            label_lists["rev_" + rev_label].append(sentence[rev_label])
    for reb_label in REB_LABELS:
        for sentence in annotation["rebuttal_sentences"]:
            label_lists["reb_" + reb_label].append(sentence[reb_label])
    return label_lists

def get_kappas_from_label_lists(anno_a, anno_b):
    kappa_map = {}
    for label, values_a in anno_a.items():
        if len(values_a) == len(anno_b[label]):
            values_b = anno_b[label]
            kappa_map[label] = kappa(values_a, values_b)
        else:
            kappa_map[label] = None
    return kappa_map


In [ ]:
# === Retrieve annotations

annotation_map = collections.defaultdict(dict)

for filename in glob.glob("../data_prep/final_dataset/test/*"):
    with open(filename, 'r') as f:
        p = json.load(f)
        annotator = get_metadata(p, "annotator")
        if annotator == "anno0":
            continue
        annotation_map[p["metadata"]["review_id"]][annotator] = p

for filename in glob.glob("../data_prep/extra_annotations/test/*"):
    with open(filename, 'r') as f:
        p = json.load(f)
        annotator = get_metadata(p, "annotator")
        if annotator == "anno0":
            continue
        annotation_map[get_metadata(p, "review_id")][get_metadata(p, "annotator")] = p
        

In [ ]:
def get_gammas(annotations):
    assert len(annotations) == 2
    continuum = Continuum()
    gammas = {}
    all_categories = set([])
    for key in "coarse fine".split():
        key_to_return = "reb_" + key
        for annotation in annotations:
            annotator = get_metadata(annotation, "annotator")
            new_continuum_items, categories = get_continuum_items_and_categories(
                annotation["rebuttal_sentences"], annotator, key)
            if not new_continuum_items:
                print("Error, no segments. Skipping this pair of annotations:", get_pair_identifier(*annotations))
                continuum = None
                break
            else:
                for item in new_continuum_items:
                    continuum.add(*item)
            all_categories = all_categories.union(categories)
        if continuum is not None:
            dissim = CombinedCategoricalDissimilarity(alpha=1, beta=2,
                        categories=list(sorted(all_categories)))
            gamma_results = continuum.compute_gamma(dissim)
            gammas[key_to_return] = gamma_results.gamma
        else:
            gammas[key_to_return] = None
    return gammas

def get_kappas(two_annotations):
    annotation_a, annotation_b = two_annotations
    kappa_map = {}

    for k, v in annotation_map.items():
        if len(v) == 1:
            continue
        else:
            for annotation_a, annotation_b in itertools.combinations(v.values(), 2):
                pair_identifier = get_pair_identifier(annotation_a, annotation_b)
                label_lists_a = get_label_lists(annotation_a)
                label_lists_b = get_label_lists(annotation_b)
                kappas = get_kappas_from_label_lists(label_lists_a,
                                                    label_lists_b)
                for k, v in kappas.items():
                    kappa_map[k] = v
    return kappa_map

In [ ]:
overall_agreements_map = {}
    
for k, v in annotation_map.items():
    if len(v) == 1:
        continue
    else:
        for two_annotations in itertools.combinations(v.values(), 2): # TODO(TJO): Is this correct for kappa? Or should we only use the two most reliable and leave the rest out of the dataset?
            # TODO(TJO): Also, I guess we shouldn't do pairwise for gamma?
            agreement_map = get_kappas(two_annotations)
            agreement_map.update(get_gammas(two_annotations))
            overall_agreements_map[get_pair_identifier(*two_annotations)] = agreement_map
    if len(overall_agreements_map) >= 5:
        break
        
print(json.dumps(overall_agreements_map))